In [41]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [42]:
segment_size = 100

# 1727 rows
data = pd.read_csv('./Dataset/STB.csv')

# Calculate the number of segments in the dataset
print("len(data)", len(data))

data['ma_5'] = data['close_price'].ewm(span=3, adjust=False, min_periods=0).mean()

# Xóa bỏ các dòng có giá trị null do moving average
data.dropna(inplace=True)


len(data) 1727


In [43]:
# closedf = data[['trunc_time','ma_5']]
closedf = data[['trunc_time','close_price','ma_5']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)

      trunc_time  close_price          ma_5
0     2016-01-04        12600  12600.000000
1     2016-01-05        12300  12450.000000
2     2016-01-06        12800  12625.000000
3     2016-01-07        12600  12612.500000
4     2016-01-08        12600  12606.250000
...          ...          ...           ...
1722  2022-11-21        16700  16748.526427
1723  2022-11-22        16900  16824.263213
1724  2022-11-23        17500  17162.131607
1725  2022-11-24        18200  17681.065803
1726  2022-11-25        18900  18290.532902

[1727 rows x 3 columns]
Shape of close dataframe: (1727, 3)


In [44]:
import copy

close_stock = closedf.copy()
del closedf['trunc_time']

X_data = copy.deepcopy(closedf)
Y_data = copy.deepcopy(closedf)

del X_data['close_price']
del Y_data['ma_5']

scaler=MinMaxScaler(feature_range=(0,1))

X_data=scaler.fit_transform(np.array(X_data).reshape(-1,1))
Y_data=scaler.fit_transform(np.array(Y_data).reshape(-1,1))

print(X_data, X_data.shape)
print(Y_data, Y_data.shape)

[[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.34368504]
 [0.36209637]
 [0.38371974]] (1727, 1)
[[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.35726795]
 [0.38178634]
 [0.40630473]] (1727, 1)


In [45]:
del closedf['close_price']
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

print(closedf.shape)

(1727, 1)


In [46]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
# train_data

train_data:  (1381, 1)
test_data:  (346, 1)


In [47]:

def create_dataset(data_set):
    training_size=int(len(data_set)*0.8)
    train_data,test_data=data_set[0:training_size,:],data_set[training_size:len(data_set),:1]
    print("train_data: ", train_data.shape)
    print("test_data: ", test_data.shape)
    return train_data,test_data

In [48]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10

X_train, X_test = create_dataset(X_data)
y_train, y_test = create_dataset(Y_data)
print("X_train: ", X_train)
print("y_train: ", y_train)
print("X_test: ", X_test)
print("y_test", y_test)


train_data:  (1381, 1)
test_data:  (346, 1)
train_data:  (1381, 1)
test_data:  (346, 1)
X_train:  [[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.82183932]
 [0.79453479]
 [0.76846483]]
y_train:  [[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.80210158]
 [0.76357268]
 [0.73905429]]
X_test:  [[0.76075172]
 [0.73383373]
 [0.74521013]
 [0.74468948]
 [0.72757657]
 [0.72966385]
 [0.7262726 ]
 [0.72723791]
 [0.74013826]
 [0.74658844]
 [0.75690935]
 [0.7585219 ]
 [0.766424  ]
 [0.78101878]
 [0.77767244]
 [0.78486905]
 [0.80354599]
 [0.81998028]
 [0.81489276]
 [0.81944482]
 [0.81994689]
 [0.81665002]
 [0.80080993]
 [0.79466384]
 [0.80223453]
 [0.80335895]
 [0.80037324]
 [0.79888038]
 [0.76886367]
 [0.73079388]
 [0.71619387]
 [0.71066783]
 [0.70346991]
 [0.70341887]
 [0.71403709]
 [0.70958944]
 [0.70470467]
 [0.70847114]
 [0.71656322]
 [0.70996553]
 [0.70755365]
 [0.70368678]
 [0.69643148]
 [0.68836893]
 [0.68345068]
 [0.68365249]
 [0.68464038]
 [0.69045619]
 [0.68804222]
 [0.68506128]
 [0

In [49]:
class Node:
    def __init__(self, split_feature=None, split_value=None, left=None, right=None, prediction=None, is_leaf=False):
        self.split_feature = split_feature
        self.split_value = split_value
        self.left = left
        self.right = right
        self.prediction = prediction
        self.is_leaf = is_leaf

In [50]:
import numpy as np
from joblib import Parallel, delayed
from sklearn.metrics import r2_score

class RandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, subsample_ratio=0.8, n_jobs=-1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.subsample_ratio = subsample_ratio
        self.trees = []
        self.feature_importances_ = None
        self.n_jobs = n_jobs
        
    def fit(self, X, y):
        n_samples = len(X)
        n_subsample = int(self.subsample_ratio * n_samples)
        self.feature_importances_ = np.zeros(X.shape[1])
        
        def fit_tree(i):
            subsample_indices, oob_indices = self._bootstrap_sampling(n_samples, n_subsample)  # Apply bootstrap sampling
            subsample_X = X[subsample_indices]
            subsample_y = y[subsample_indices]
            
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                         min_samples_leaf=self.min_samples_leaf, max_features=self.max_features)
            tree.fit(subsample_X, subsample_y)
            
            return tree, subsample_indices
        
        results = Parallel(n_jobs=self.n_jobs)(delayed(fit_tree)(i) for i in range(self.n_estimators))
        
        self.trees = [result[0] for result in results]
        self.feature_importances_ = np.sum([tree.feature_importances_ for tree, _ in results], axis=0)
        
    def predict(self, X):
        return np.mean([tree.predict(X) for tree in self.trees], axis=0)
    
    def _bootstrap_sampling(self, n_samples, n_subsample):
        subsample_indices = np.random.choice(n_samples, size=n_subsample, replace=True)  # Subsampling with Replacement
        oob_indices = np.array(list(set(range(n_samples)) - set(subsample_indices)))  # Out-of-Bag indices
        return subsample_indices, oob_indices


class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.tree = None
        self.feature_importances_ = None
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        self.feature_importances_ = self._calculate_feature_importances(X)
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _build_tree(self, X, y, depth=0):
        node = Node()
        
        if depth == self.max_depth or len(X) < self.min_samples_split or len(np.unique(y)) == 1:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node
        
        split_feature, split_value = self._find_best_split(X, y)
        
        if split_feature is None:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node
        
        left_indices = X[:, split_feature] <= split_value
        right_indices = X[:, split_feature] > split_value

        node.split_feature = split_feature
        node.split_value = split_value

        node.left = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        node.right = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return node
    
    def _find_best_split(self, X, y):
        best_score = float('inf')
        best_split_feature = None
        best_split_value = None
        
        n_features = X.shape[1]
        if self.max_features is not None and self.max_features < n_features:
            feature_indices = np.random.choice(n_features, size=self.max_features, replace=False)
        else:
            feature_indices = np.arange(n_features)
        
        for feature in feature_indices:
            unique_values = np.unique(X[:, feature])
            split_values = (unique_values[:-1] + unique_values[1:]) / 2.0
            
            for value in split_values:
                left_indices = X[:, feature] <= value
                right_indices = X[:, feature] > value
                score = self._calculate_split_score(y, left_indices, right_indices)
                
                if score < best_score:
                    best_score = score
                    best_split_feature = feature
                    best_split_value = value
        
        return best_split_feature, best_split_value
    
    def _calculate_split_score(self, y, left_indices, right_indices):
        left_y = y[left_indices]
        right_y = y[right_indices]
        
        left_score = self._calculate_variance(left_y)
        right_score = self._calculate_variance(right_y)
        
        n_left = len(left_y)
        n_right = len(right_y)
        total_samples = n_left + n_right
        
        split_score = (n_left / total_samples) * left_score + (n_right / total_samples) * right_score
        return split_score
    
    def _calculate_variance(self, y):
        if len(y) == 0:
            return 0.0
        mean = np.mean(y)
        variance = np.mean((y - mean) ** 2)
        return variance
    
    def _traverse_tree(self, x, node):
        if node.is_leaf:
            return node.prediction
        else:
            if x[node.split_feature] <= node.split_value:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)
            
    def _calculate_feature_importances(self, X):
        feature_importances = np.zeros(X.shape[1])  # Adjust the shape of feature_importances array
        # Calculate feature importances based on tree structure, node splits, or other criteria
        # Assign importance values to each feature
        return feature_importances


class Node:
    def __init__(self):
        self.is_leaf = False
        self.prediction = None
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None


In [51]:
from sklearn.metrics import mean_squared_error
import optuna
# from sklearn.ensemble import RandomForestRegressor as SklearnRandom

# Define the objective function for Optuna
# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 10, 1000)
#     max_depth = trial.suggest_int('max_depth', 2, 32)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
#     max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
#     print(n_estimators)

#     model = RandomForestRegressor(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features    
#     )

#     print(n_estimators)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)

#     return mse

# # Use Optuna to optimize hyperparameters
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=2)
# best_params = study.best_params

In [52]:
rf = RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,n_estimators=160, max_features=1)
# rf.fit(X_train, y_train)

In [53]:
# rf = SklearnRandom(max_depth=26, min_samples_leaf=2, min_samples_split=10,
#                       n_estimators=160)
rf.fit(X_train, y_train)

In [54]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)

RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1381, 1)
Test data prediction: (346, 1)


In [55]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [56]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  145.70860289573548
Train data MSE:  21230.996957827134
Test data MAE:  95.27522607749955
-------------------------------------------------------------------------------------
Test data RMSE:  805.3725266080896
Test data MSE:  648624.906615098
Test data MAE:  594.4579630330026


In [57]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9987156433249367
Test data explained variance regression score: 0.9751495684807193


In [58]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9987156346239531
Test data R2 score: 0.9744895561531647


In [59]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  9.631026230646e-05
Test data MGD:  0.0008307052525652587
----------------------------------------------------------------------
Train data MPD:  1.3289729789718878
Test data MPD:  22.753948035923266


In [60]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px

print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(closedf.shape)
print(len(close_stock))

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[0:len(RF_train_predict), :] = RF_train_predict

print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(RF_train_predict):len(closedf), :] = RF_test_predict

print("Test predicted data: ", testPredictPlot.shape)

close_stock['Predictions']=testPredictPlot
close_stock['Stock']='STB'
close_stock['Model']='DIY'
close_stock['Method']='MV'
close_stock.to_csv('./output/STB_DIY_MV.csv', index=False)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['trunc_time'],
                       'original_close': close_stock['ma_5'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1381, 1)
(346, 1)
(1727, 1)
1727
Train predicted data:  [[12726.49191394]
 [12484.99928252]
 [12858.61243679]
 ...
 [           nan]
 [           nan]
 [           nan]]
Test predicted data:  (1727, 1)
